In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Descenso gradiente

Vamos a usar el dataset de propiedades en Boston (https://www.kaggle.com/c/boston-housing) y tratar de predecir el valor de `medv` usando una regresión lineal múltiple.

Para eso, vamos a modificar la clase `MyGradientDescent` presentada en el encuentro sincrónico y usarla para entrenar un modelo de regresión lineal múltiple y uno simple.

Ayuda: 

<p style="font-size:16px;">
$h =  \beta_0 + \beta_1. X_1 + \beta_2. X_2 + \beta_3. X_3 + ... + \beta_m. X_m$
</p>

i es el índice de la fila en el dataset

<p style="font-size:16px;">
$h_i =  \beta_0 + \beta_1. X_{i1} + \beta_2. X_{i2} + \beta_3. X_{i3} + ... + \beta_m. X_{im}$
</p>    

Update: 

<p style="font-size:16px;">
$\beta_0 = \beta_0 - \alpha \frac{1}{N} \sum (h_i - y_i)$
</p>    
<p style="font-size:16px;">
$\beta_i = \beta_i - \alpha \frac{1}{N} \sum (h_i - y_i). X_i$
</p>    

Costo (error cuadrático medio): 
<p style="font-size:16px;">    
$J(\beta_0, ..., \beta_m) = \frac{1}{N} \sum_{i=1}^N (h_i - y_i)^2  $
</p>
Gradiente: 

<p style="font-size:16px;">    
$\frac{\partial J(\beta_0, ..., \beta_m)}{\partial \beta_j} = \frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{ij} $
</p>    

N es el número de observaciones o filas del dataset

Entonces 

$\beta_0 = \beta_0 - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{i0}$

como $X_{i0} = 1$ queda:

$\beta_0 = \beta_0 - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i)$

$\beta_1 = \beta_1 - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{i1}$

$\beta_2 = \beta_2 - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{i2}$

... 

$\beta_j = \beta_j - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{ij}$


$\alpha$ = Learning Rate

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

## Ejercicio 1

Leer los datos del archivo `Data/boston_data.csv` en un dataframe y construir un heatmap de correlaciones entre sus columnas

In [ ]:
data = pd.read_csv('../Data/boston_data.csv')

data.head()

In [ ]:
sns.heatmap(data.corr());

## Ejercicio 2 

La variable target del modelo es `medv`.

Seleccionar como variables predictoras las tres variables que tengan mayor correlación (en valor absoluto) con la variable target.

Construir los conjuntos de train y test y normalizar las features.

In [ ]:
abs(data.corr()['medv']).sort_values()

In [ ]:
features = ['lstat', 'rm', 'ptratio']
X_features = data[features]
y = data['medv']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, random_state = 12)

In [ ]:
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

## Ejercicio 3

Modificar la clase `MyGradientDescent` presentada en el encuentro sincrónico para resolver ahora una regresión **múltiple** usando descenso gradiente.

```

class MyGradientDescent():
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate
        self.beta1 = 0
        self.beta0 = 0
          
    def fit(self, X, y, epochs = 100):
        N = len(X)
        history = []
        
        for e in range(epochs):
            for i in range(N):
                Xi = X[i, :]
                yi = y.iloc[i]                 
                
                hi = self.beta1 * Xi + self.beta0
                f = hi - yi
                
                self.beta1 -= self.learning_rate * 2 / N * f * Xi
                self.beta0 -= self.learning_rate * 2 / N * f 

            loss = 0
            loss = mean_squared_error(y, (self.beta1 * X + self.beta0))
                                      
            if e % 100 == 0:
                print(f"Epoch: {e}, Loss: {loss})")
            
            history.append(loss)
                                      
        return history
                
    def predict(self, X):
        return self.beta1 * X + self.beta0


```

Tener en cuenta las fórmulas presentadas en la ayuda al inicio de la notebook.

In [ ]:
class MyGradientDescentMultiple():
    
    def __init__(self, learning_rate, m):
        self.learning_rate = learning_rate
        self.betas = np.repeat(0, m)
        self.beta0 = 0
          
    def fit(self, X, y, epochs = 100):
        N = len(X)
        m = X.shape[1] 
        history = []
        
        for e in range(epochs):
            gradiente_0 = 0
            gradiente = np.repeat(0, m)
            for i in range(N):
                Xi = X[i, :]
                yi = y.iloc[i]                 

                hi = np.dot(Xi, self.betas) + self.beta0    
                gradiente = gradiente + (hi - yi) * Xi
                gradiente_0 = gradiente_0 + (hi - yi)


            self.beta0 = self.beta0 - self.learning_rate * 2 / N * gradiente_0   
            self.betas = self.betas - self.learning_rate * 2 / N * gradiente
                    
            pred = np.dot(X, self.betas) + self.beta0
            loss = mean_squared_error(y, pred)
                                      
            if e % 100 == 0:
                print(f"Epoch: {e}, Loss: {loss})")
            
            history.append(loss)
                                      
        return history
            
    def predict(self, X):
        return np.dot(X, self.betas) + self.beta0

## Ejercicio 4

Entrenar la regresión lineal múltiple con tres variables predictoras usando la clase que definieron en el ejercicio 3.

Evaluar la performance en test mediante el error cuadrático medio.

In [ ]:
model = MyGradientDescentMultiple(learning_rate = 0.01, m = 3)
history = model.fit(X_train_scl, y_train, 1000)

predictions = model.predict(X_test_scl)

In [ ]:
mean_squared_error(y_test, predictions)

## Ejercicio 5

Graficar el valor de pérdida en función de las épocas

In [ ]:
sns.lineplot(x = range(len(history)), y = history);

## Ejercicio 6

Usar la misma clase del ejercicio 3 para ajustar una regresión lineal simple cuya variable predictora sea `lstat` y comprobar que esta clase da el mismo resultado que `MyGradientDescent`

Graficar en un scatterplot los datos de test y los predichos por el modelo.

In [ ]:
features = ['lstat']
X_features = data[features]
y = data['medv']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, random_state = 12)

In [ ]:
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

In [ ]:
model_simple = MyGradientDescentMultiple(learning_rate = 0.01, m = 1)
history = model_simple.fit(X_train_scl, y_train, 1000)

predictions = model_simple.predict(X_test_scl)

In [ ]:
print(model_simple.betas)
print(model_simple.beta0)

In [ ]:
mean_squared_error(y_test, predictions)

In [ ]:
sns.lineplot(x = range(len(history)), y = history);

In [ ]:
sns.scatterplot(x = X_test_scl[:, 0], y = y_test )
sns.lineplot(x = X_test_scl[:, 0], y = predictions, color="orange");

Ahora veamos que devuelve `MyGradientDescent`

In [ ]:
class MyGradientDescent():
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate
        self.beta1 = 0
        self.beta0 = 0
          
    def fit(self, X, y, epochs = 100):
        N = len(X)
        history = []
        
        for e in range(epochs):
            for i in range(N):
                Xi = X[i, :]
                yi = y.iloc[i] 
                
                hi = self.beta1 * Xi + self.beta0
                f = hi - yi
                
                self.beta1 -= self.learning_rate * 2 / N * f * Xi
                self.beta0 -= self.learning_rate * 2 / N * f 

            loss = 0
            loss = mean_squared_error(y, (self.beta1 * X + self.beta0))
                                      
            if e % 100 == 0:
                print(f"Epoch: {e}, Loss: {loss})")
            
            history.append(loss)
                                      
        return history
                
    def predict(self, X):
        return self.beta1 * X + self.beta0

In [ ]:
model_lineal_simple = MyGradientDescent(learning_rate = 0.01)
history = model_lineal_simple.fit(X_train_scl, y_train, 1000)

predictions = model_lineal_simple.predict(X_test_scl)

In [ ]:
print(model_lineal_simple.beta1)
print(model_lineal_simple.beta0)

In [ ]:
mean_squared_error(y_test, predictions)

## Ejercicio 7 - Opcional

Intenten entrenar un modelo con cinco variables predictoras. 

Posiblemente tengan que probar distintos valores de learning rate para conseguir resultados aceptables.

## Conclusión

Implementando de forma más general la clase `MyGradientDescent` logramos usar el mismo código para resolver regresiones lineales simples y múltiples con descenso gradiente.

## Referencias

---

https://towardsdatascience.com/multivariate-linear-regression-in-python-step-by-step-128c2b127171

https://towardsdatascience.com/gradient-descent-in-python-a0d07285742f
